In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))

In [2]:
import ee 
import geemap

ee.Initialize(project="thurgau-irrigation")

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [3]:
from src.et_green.compute_et_green import compute_et_green, calculate_band_std_dev
from src.et_green.filter_nutzungsflaechen import (
    get_crops_to_exclude,
    get_rainfed_reference_crops,
    create_crop_filters,
    filter_crops,
    add_double_cropping_info,
    get_unique_nutzung,
)
from src.et_green.exporting_utils import process_et_green, prepare_rainfed_fields

from utils.ee_utils import (
    back_to_float,
    back_to_int,
    export_image_to_asset,
    print_value_ranges,
)

from utils.date_utils import print_collection_dates

---

## Constants

In [4]:
YEAR = 2021

PATH_TO_AOI = "projects/thurgau-irrigation/assets/Thurgau/thrugau_borders_2024"
PATH_TO_ET_PRODUCT = (
    f"projects/thurgau-irrigation/assets/ET_Landsat_decadal_canton_v2"
)
PATH_TO_DOUBLE_CROPPING_COLLECTION = f"projects/thurgau-irrigation/assets/Thurgau/VegetationPeriod/crop_veg_period_{YEAR}"
PATH_TO_JURISDICTION = (
    f"projects/thurgau-irrigation/assets/Thurgau/thurgau_jurisdictions_{YEAR}_wgs84"
)
PATH_TO_LANDUSE = (
    f"projects/thurgau-irrigation/assets/Thurgau/Nutzungsflaechen/TG_{YEAR}_area"
)
LANDUSE_PROPERTY_NAME = "nutzung"

ET_BAND_NAME = "ET"
SCALING_FACTOR = 10  # scaling factor for the ET band to convert it back to float
DYNAMIC = True
SCALING_FACTOR_PROPERTY_NAME = "days"
ET_BAND_RESOLUTION = 10  # in meters
NUMBER_OF_IMAGES = 36
TEMPORAL_RESOLUTION = "dekadal"


ET_GREEN_ASSET_PATH = f"projects/thurgau-irrigation/assets/Thurgau/ET_green/ET_green_{YEAR}_{TEMPORAL_RESOLUTION}_from_WaPOR_10m"

## 1. Load Assets

In [5]:
aoi = ee.FeatureCollection(PATH_TO_AOI).geometry().simplify(500).buffer(100)

In [6]:
double_cropping_image = ee.Image(PATH_TO_DOUBLE_CROPPING_COLLECTION)

In [7]:
et_collection = (
    ee.ImageCollection(PATH_TO_ET_PRODUCT)
    .filterBounds(aoi)
    .filterDate(f"{YEAR}-01-01", f"{YEAR}-12-31")
    .map(lambda img: back_to_float(img, SCALING_FACTOR))
)

et_collection_list = et_collection.toList(et_collection.size())


# Sanity checks:
print_collection_dates(et_collection)

Dates of images in the collection:
2021-04-01
2021-04-11
2021-04-21
2021-06-11
2021-06-21
2021-07-01
2021-07-21
2021-08-11
2021-09-01
2021-09-21
2021-10-01
2021-04-01
2021-04-11
2021-04-21
2021-05-01
2021-06-11
2021-06-21
2021-07-21
2021-08-11
2021-09-01
2021-09-21
2021-10-01


In [8]:
jurisdictions = ee.FeatureCollection(PATH_TO_JURISDICTION)

In [9]:
landuse_collection = ee.FeatureCollection(PATH_TO_LANDUSE)

# Check if the collection has "nutzung" property, otherwise rename it
properties = landuse_collection.first().propertyNames().getInfo()

landuse_collection = (
    landuse_collection.map(lambda f: f.set("nutzung", f.get(LANDUSE_PROPERTY_NAME)))
    if LANDUSE_PROPERTY_NAME != "nutzung"
    else landuse_collection
)

print(
    f"Renamed {LANDUSE_PROPERTY_NAME} to 'nutzung'"
    if LANDUSE_PROPERTY_NAME != "nutzung"
    else "Collection has 'nutzung' property"
)

Collection has 'nutzung' property


## 2. Compute ET_green

In [10]:
# process_et_green(
#     et_collection_list=et_collection_list,
#     landuse_collection=landuse_collection,
#     jurisdictions=jurisdictions,
#     double_cropping_image=double_cropping_image,
#     year=YEAR,
#     aoi=aoi,
#     asset_path=ET_GREEN_ASSET_PATH,
#     et_band_name=ET_BAND_NAME,
#     time_steps=NUMBER_OF_IMAGES,
#     time_step_type=TEMPORAL_RESOLUTION,
#     resolution=ET_BAND_RESOLUTION,
# )

In [11]:
et_green_list = []


not_irrigated_crops = get_crops_to_exclude()
rainfed_crops = get_rainfed_reference_crops()

# Prepare rainfed fields
rainfed_fields = prepare_rainfed_fields(
    landuse_collection, double_cropping_image, not_irrigated_crops, rainfed_crops
)

for img in range(NUMBER_OF_IMAGES):
    et_img = ee.Image(et_collection_list.get(img))

    et_green_img = compute_et_green(
        et_img, rainfed_fields, jurisdictions, et_band_name=ET_BAND_NAME
    )
    et_green_list.append(et_green_img)

In [12]:
# for image in et_green_list:
#     print(f"Date: {ee.Image(image).date().format('YYYY-MM-dd').getInfo()}")

# # print(f"Number of images: {len(et_green_list)}")

In [14]:
Map = geemap.Map()

image = ee.Image(et_green_list[4])
# image_ET = ee.Image(et_collection_list.get(20))

vis_params = {
    "bands": ["ET_green"],
    "min": 0,
    "max": 5,
    "palette": ["blue", "lightblue", "green", "yellow", "orange", "red"],
}

vis_params_ET = {
    "bands": [ET_BAND_NAME],
    "min": 0,
    "max": 5,
    "palette": ["blue", "lightblue", "green", "yellow", "orange", "red"],
}



Map.center_object(aoi, 12)
Map.addLayer(image, vis_params, "ET green 10m")
Map.add_colorbar(vis_params, label="ET green [mm/month]")
# Map.addLayer(image_ET, vis_params_ET, "ET 10m")


Map

Map(center=[47.56858787382066, 9.092720596553875], controls=(WidgetControl(options=['position', 'transparent_b…